## Script 9, Saloni Khandelwal
## Summarization: Articles Positive and Negative

In [ ]:
import pandas as pd
import spacy
from collections import Counter

In [ ]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

In [ ]:
df = pd.read_parquet('entity_nlp.parquet')
df.head()

,url,date,language,title,text,cleaned_text,important_words,tokens,topic,predicted_sentiment_yelp_new,sentiment_numeric,entities,organization,products,GPE
0,http://en.people.cn/n3/2021/0318/c90000-983012...,2021-03-18,en,Artificial intelligence improves parking effic...,\n\nArtificial intelligence improves parking e...,Some urban areas of the city started to use ET...,urban areas city started use etc system roadsi...,"[urban, areas, city, started, use, etc, system...",0,p,1,"[[ETC, ORG], [July 1, 2019, DATE], [ETC, ORG],...","[ETC, ETC, AI, ETC, Wang, ETC, AIpark, AI, AI,...",[],"[Chaoyang, Beijing, Beijing, Beijing, China, C..."
1,http://newsparliament.com/2020/02/27/children-...,2020-02-27,en,Children With Autism Saw Their Learning and So...,\nChildren With Autism Saw Their Learning and ...,Children With Autism Saw Their Learning and So...,children autism saw learning social skills boo...,"[children, autism, saw, learning, social, skil...",1,n,-1,"[[Thursday, February 27, 2020, DATE], [seven, ...","[Kiwi, Kiwi, Kiwi, Science Robotics, Kiwi, Kiw...",[Cancel],"[U.S., Matarić]"
2,http://www.dataweek.co.za/12835r,2021-03-26,en,"Forget ML, AI and Industry 4.0 – obsolescence ...","\n\nForget ML, AI and Industry 4.0 – obsolesce...",The world entered a new era of accelerated tra...,world entered new era accelerated transformati...,"[world, entered, new, era, accelerated, transf...",0,p,1,"[[the last eighteen months, DATE], [years, DAT...","[the Brazilian Embassy, PowerPoint, ML, AI, Ir...",[],"[London, Lego, Mexico, US]"
3,http://www.homeoffice.consumerelectronicsnet.c...,2021-03-10,en,Strategy Analytics: 71% of Smartphones Sold Gl...,\n\nStrategy Analytics: 71% of Smartphones Sol...,AI is used in various functions inside smartph...,ai used various functions inside smartphones i...,"[ai, used, various, functions, inside, smartph...",0,p,1,"[[AI, ORG], [AI, ORG], [AI, ORG], [Edge AI, OR...","[AI, AI, AI, Edge AI, AI, AI, AI, AI, AI, AI, ...",[],[]
4,http://www.itbusinessnet.com/2020/10/olympus-t...,2020-10-20,en,Olympus to Support Endoscopic AI Diagnosis Edu...,\n\nOlympus to Support Endoscopic AI Diagnosis...,"In collaboration with CYBERNET SYSTEMS CO,. LT...",collaboration cybernet systems co ltd olympus ...,"[collaboration, cybernet, systems, co, ltd, ol...",0,p,1,"[[CYBERNET SYSTEMS CO, ORG], [Olympus, ORG], [...","[CYBERNET SYSTEMS CO, Olympus, AI, the Asian I...",[],"[Hyderabad, India, India, Japan, India, India,..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 136927 entries, 0 to 200331
Data columns (total 15 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   url                           136927 non-null  object        
 1   date                          136927 non-null  datetime64[ns]
 2   language                      136927 non-null  object        
 3   title                         136927 non-null  object        
 4   text                          136927 non-null  object        
 5   cleaned_text                  136927 non-null  object        
 6   important_words               136927 non-null  object        
 7   tokens                        136927 non-null  object        
 8   topic                         136927 non-null  int64         
 9   predicted_sentiment_yelp_new  136927 non-null  object        
 10  sentiment_numeric             136927 non-null  int64         
 11  entities          

In [ ]:
type(df['entities'])

pandas.core.series.Series

In [ ]:
df['entities'] = df['entities'].astype(str)

In [ ]:
p_df = df[df['predicted_sentiment_yelp_new'] == 'p']
n_df = df[df['predicted_sentiment_yelp_new'] == 'n']

In [ ]:
summarize_pos_df = p_df[p_df['entities'].str.contains('ChatGPT')]
summarize_pos_df.head()

,url,date,language,title,text,cleaned_text,important_words,tokens,topic,predicted_sentiment_yelp_new,sentiment_numeric,entities,organization,products,GPE
88,https://finance.yahoo.com/news/sensorium-lead-...,2023-02-02,en,Sensorium to Lead Conversation on AI Virtual B...,Sensorium to Lead Conversation on AI Virtual B...,"""It's fantastic to have the opportunity to lea...",fantastic opportunity lead conversation ai aid...,"[fantastic, opportunity, lead, conversation, a...",0,p,1,"[array(['AI', 'ORG'], dtype=object) array(['LE...","[AI, LEAP, Sojitz, Meta, MicroVision, MicroVis...",[],"[Germany, China]"
92,https://fossbytes.com/chatgpt-ai-google-code-red/,2022-12-22,en,"""Code Red"": ChatGPT AI Scares Google Into Acti...","\n\n""Code Red"": ChatGPT AI Scares Google Into ...","Priye Rai December 22, 2022 Tweet Share WhatsA...",priye rai december 22 2022 tweet share whatsap...,"[priye, rai, december, 22, 2022, tweet, share,...",1,p,1,"[array(['Rai December 22', 'DATE'], dtype=obje...","[AI, AI, Google, The New York Times, Google, A...",[],"[Fossbytes, Us]"
313,https://www.binance.com/en/news/flash/7393767,2023-02-23,en,Best 5 Ways ChatGPT Can Help Blockchain Develo...,\n\nBest 5 Ways ChatGPT Can Help Blockchain De...,We do not endorse their accuracy. Digital asse...,endorse accuracy digital asset prices volatile...,"[endorse, accuracy, digital, asset, prices, vo...",0,p,1,"[array(['CoinGape', 'ORG'], dtype=object)\n ar...","[CoinGape, ChatGPT, ResearchChatGPT, creationC...",[],[Blockchain]
325,https://www.breakinglatest.news/business/lectu...,2023-02-14,en,Lecture appointment丨Six authoritative experts ...,\nLecture appointment丨Six authoritative expert...,Lecture appointment丨Six authoritative experts ...,lecture appointment丨six authoritative experts ...,"[lecture, appointment丨six, authoritative, expe...",0,p,1,"[array(['AI', 'ORG'], dtype=object)\n array(['...","[AI, Latest News Home Business Entertainment H...",[BuzzFeed],"[US, Berkeley, California, Tianlang, China, th..."
386,https://www.dailymail.co.uk/travel/article-119...,2023-04-05,en,We asked ChatGPT and Google's Bard to plan a v...,We asked ChatGPT and Google's Bard to plan a v...,Tour from Istanbul to London across 22 countri...,tour istanbul london across 22 countries 56 da...,"[tour, istanbul, london, across, 22, countries...",1,p,1,"[array(['Istanbul', 'GPE'], dtype=object)\n ar...","[Showtime, MailOnline Travel, Google, Bard, Ba...","[Kilimanjaro, Shard]","[Istanbul, London, Lapland, Amsterdam, Paris, ..."


In [ ]:
summarize_pos_df['cleaned_text']

88        "It's fantastic to have the opportunity to lea...
92        Priye Rai December 22, 2022 Tweet Share WhatsA...
313       We do not endorse their accuracy. Digital asse...
325       Lecture appointment丨Six authoritative experts ...
386       Tour from Istanbul to London across 22 countri...
                                ...                        
199815    Codie.ai, first consultative AI assistant base...
199866    ChatGPT is one of the latest additions to the ...
199981    This story about AI was written by a human: Ho...
200142    We may be compensated by vendors who appear on...
200158    As a woman who's been single but dating for al...
Name: cleaned_text, Length: 1778, dtype: object

In [ ]:
pos_text = str(summarize_pos_df['cleaned_text'].tolist())
pos_text[:390]

'[\'"It\\\'s fantastic to have the opportunity to lead the conversation on AI and AI-driven virtual beings at a stage like LEAP, which has become a global window into the state of emerging technology and the most exciting advancements the industry is experiencing. Over the last year, we\\\'ve continued the development of our pioneering AI-driven avatars and have released groundbreaking metaver'

In [ ]:
type(pos_text)

str

In [ ]:
from summarizer import Summarizer

In [ ]:
type(pos_text[:390])

str

In [ ]:
model = Summarizer()

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
result = model(pos_text[:300], ratio=0.1)  # Specified with ratio
result

'[\'"It\\\'s fantastic to have the opportunity to lead the conversation on AI and AI-driven virtual beings at a stage like LEAP, which has become a global window into the state of emerging technology and the most exciting advancements the industry is experiencing.'

#### exciting advancements the industry is experiencing.

In [ ]:
filtered_df = summarize_pos_df[summarize_pos_df['cleaned_text'].str.contains('ChatGPT') & df['cleaned_text'].str.contains('exciting adv')]
filtered_df

/var/folders/1q/cr3dj0m115vf89fcxv59dk7r0000gn/T/ipykernel_6262/1090423295.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_df = summarize_pos_df[summarize_pos_df['cleaned_text'].str.contains('ChatGPT') & df['cleaned_text'].str.contains('exciting adv')]


,url,date,language,title,text,cleaned_text,important_words,tokens,topic,predicted_sentiment_yelp_new,sentiment_numeric,entities,organization,products,GPE
88,https://finance.yahoo.com/news/sensorium-lead-...,2023-02-02,en,Sensorium to Lead Conversation on AI Virtual B...,Sensorium to Lead Conversation on AI Virtual B...,"""It's fantastic to have the opportunity to lea...",fantastic opportunity lead conversation ai aid...,"[fantastic, opportunity, lead, conversation, a...",0,p,1,"[array(['AI', 'ORG'], dtype=object) array(['LE...","[AI, LEAP, Sojitz, Meta, MicroVision, MicroVis...",[],"[Germany, China]"


In [ ]:
x = filtered_df.iloc[0]['url']
print(x)

https://finance.yahoo.com/news/sensorium-lead-conversation-ai-virtual-110000531.html


#### Finanace Yahoo Domain

#### Title
Sensorium to Lead Conversation on AI Virtual Beings at LEAP 2023

#### Text
It's fantastic to have the opportunity to lead the conversation on AI and AI-driven virtual beings at a stage like LEAP, which has become a global window into the state of emerging technology and the most exciting advancements the industry is experiencing

In [ ]:
len(pos_text)

9429703

In [ ]:
sliced_text = pos_text[9429503:]
sliced_text

"ChatGPT gave me made me seem more mature, charitable, and hospitable. I still received a good amount of likes based on my photos, so were my pictures so good that men didn\\'t even care what I wrote?']"

In [ ]:
summarizer = Summarizer()
summary = summarizer( pos_text[9429603:])
print(summary)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


of likes based on my photos, so were my pictures so good that men didn\'t even care what I wrote?']


In [ ]:
summarize_neg_df = n_df[n_df['entities'].str.contains('ChatGPT')]
summarize_neg_df.head()

,url,date,language,title,text,cleaned_text,important_words,tokens,topic,predicted_sentiment_yelp_new,sentiment_numeric,entities,organization,products,GPE
48,https://centralpointnews.com/recent-chatgpt-an...,2023-04-06,en,Recent ChatGPT And Bard Predicament Raises Tho...,\nRecent ChatGPT And Bard Predicament Raises T...,"Maybe that’s true, possibly not. One point tha...",maybe thats true possibly one point cognize ju...,"[maybe, thats, true, possibly, one, point, cog...",1,n,-1,"[array(['One', 'CARDINAL'], dtype=object)\n ar...","[Artificial Intelligence, AI, AI apps arsenic,...",[],"[U.S., OpenAI, OpenAI, OpenAI, OpenAI, OpenAI,..."
232,https://techstory.in/chatgpt-marks-elon-narend...,2023-02-20,en,ChatGPT marks Elon & Narendra Modi as a 'contr...,\n\nChatGPT marks Elon & Narendra Modi as a 'c...,ChatGPT marks Elon & Narendra Modi as a 'contr...,chatgpt marks elon narendra modi controversial...,"[chatgpt, marks, elon, narendra, modi, controv...",1,n,-1,"[array(['ChatGPT marks Elon & Narendra Modi', ...","[ChatGPT marks Elon & Narendra Modi, Issac Lat...","[Twitter, NewsTech Binance]","[Washington, U.S., Russia, the United Kingdom,..."
269,https://vestaviavoice.com/news/student-or-bot-...,2023-03-01,en,"Student or bot? ChatGPT, other tools leading t...","\n\n\nStudent or bot? ChatGPT, other tools lea...","ChatGPT, other tools leading to new conversati...",chatgpt tools leading new conversations vestav...,"[chatgpt, tools, leading, new, conversations, ...",1,n,-1,"[array(['ChatGPT', 'ORG'], dtype=object)\n arr...","[ChatGPT, NEAL, ChatGPT, the Vestavia Voice, G...",[],"[Vestavia Hills City Schools, OpenAI, OpenAI, ..."
270,https://voonze.com/petey-the-new-chatgpt-app-f...,2023-03-19,en,Petey: The new ChatGPT App for Apple Watch. - ...,\nPetey: The new ChatGPT App for Apple Watch....,Petey: The new ChatGPT App for Apple Watch. - ...,petey new chatgpt app apple watch smart techno...,"[petey, new, chatgpt, app, apple, watch, smart...",1,n,-1,"[array(['App', 'PERSON'], dtype=object)\n arra...","[Apple Watch, Tech News How, Social Networks S...",[],"[OpenAI, OpenAI, OpenAI, OpenAI]"
338,https://www.businessinsider.com/elon-musk-bill...,2023-02-26,en,"What Musk, Gates, Other Business Leaders Think...","\n\nWhat Musk, Gates, Other Business Leaders T...","What Musk, Gates, Other Business Leaders Think...",musk gates business leaders think chatgpt ai j...,"[musk, gates, business, leaders, think, chatgp...",1,n,-1,"[array(['Musk', 'PERSON'], dtype=object)\n arr...","[Gates, AI Jump to Main, Chevron, Chevron, AI,...","[Email, Twitter, DeepMind]","[OpenAI, Alphabet, Alphabet, Dubai, OpenAI, Op..."


In [ ]:
neg_text = str(summarize_neg_df['cleaned_text'].tolist())
neg_text[:300]

'["Maybe that’s true, possibly not. One point that we cognize for judge is that location is simply a benignant of conflict aliases warfare taking spot among nan various Artificial Intelligence (AI) makers that are hurriedly and desperately aiming to bring their generative AI apps into nan marketplace'

In [ ]:
len(neg_text)

8379639

In [ ]:
result = model(neg_text[:300], ratio=0.1)  # Specified with ratio
result

'One point that we cognize for judge is that location is simply a benignant of conflict aliases warfare taking spot among nan various Artificial Intelligence (AI) makers that are hurriedly and desperately aiming to bring their generative AI apps into nan marketplace'

#### hurriedly and desperately aiming to bring their generative AI apps into nan marketplace

In [ ]:
filtered_df = summarize_neg_df[summarize_neg_df['cleaned_text'].str.contains('ChatGPT') & df['cleaned_text'].str.contains('hurriedly and desperately')]
filtered_df

/var/folders/1q/cr3dj0m115vf89fcxv59dk7r0000gn/T/ipykernel_6262/158262284.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_df = summarize_neg_df[summarize_neg_df['cleaned_text'].str.contains('ChatGPT') & df['cleaned_text'].str.contains('hurriedly and desperately')]


,url,date,language,title,text,cleaned_text,important_words,tokens,topic,predicted_sentiment_yelp_new,sentiment_numeric,entities,organization,products,GPE
48,https://centralpointnews.com/recent-chatgpt-an...,2023-04-06,en,Recent ChatGPT And Bard Predicament Raises Tho...,\nRecent ChatGPT And Bard Predicament Raises T...,"Maybe that’s true, possibly not. One point tha...",maybe thats true possibly one point cognize ju...,"[maybe, thats, true, possibly, one, point, cog...",1,n,-1,"[array(['One', 'CARDINAL'], dtype=object)\n ar...","[Artificial Intelligence, AI, AI apps arsenic,...",[],"[U.S., OpenAI, OpenAI, OpenAI, OpenAI, OpenAI,..."


In [ ]:
x = filtered_df.iloc[0]['url']
print(x)

https://centralpointnews.com/recent-chatgpt-and-bard-predicament-raises-thorny-questions-about-whether-using-one-ai-to-train-a-competing-ai-can-be-fair-and-square-says-ai-ethics-and-ai-law-199681.html


#### Central Point News

#### Title:
Recent ChatGPT And Bard Predicament Raises Thorny Questions About Whether Using One AI To Train A Competing AI Can Be Fair And Square, Says AI Ethics And AI Law | CENTRALPOINTNEWS

#### Text:
"One point that we cognize for judge is that location is simply a benignant of conflict aliases warfare taking spot among nan various Artificial Intelligence (AI) makers that are hurriedly and desperately aiming to bring their generative AI apps into nan marketplace."

In [ ]:
summarize_pos_df = p_df[p_df['entities'].str.contains('health')]
summarize_pos_df.head()

,url,date,language,title,text,cleaned_text,important_words,tokens,topic,predicted_sentiment_yelp_new,sentiment_numeric,entities,organization,products,GPE
182,https://newslanes.com/2022/12/23/dubai-hosts-a...,2022-12-23,en,Dubai Hosts A Regional Convening On Artificial...,\nDubai Hosts A Regional Convening On Artifici...,"Held at the Jameel Arts Centre in Dubai, the c...",held jameel arts centre dubai conference atten...,"[held, jameel, arts, centre, dubai, conference...",0,p,1,"[array(['the Jameel Arts Centre', 'FAC'], dtyp...","[the Jameel Clinic’s, COVID-19, Community Jame...",[],"[Dubai, Dubai, Saudi Arabia]"
200,https://primefeed.in/news/5479798/artificial-i...,2021-01-19,en,Artificial Intelligence (AI) Chipset Market St...,\n\nArtificial Intelligence (AI) Chipset Marke...,AI chipsets see greater opportunities in the a...,ai chipsets see greater opportunities automoti...,"[ai, chipsets, see, greater, opportunities, au...",9,p,1,"[array(['AI', 'ORG'], dtype=object) array(['AI...","[AI, AI, BFSI, healthcare & pharmaceuticals, A...",[],"[US, US, US, US, China, South Korea, US, US, S..."
309,https://www.benzinga.com/markets/cannabis/21/0...,2021-04-19,en,Launching A New Avenue For CBD Farming On 4/20...,\n\nLaunching A New Avenue For CBD Farming On ...,The near-infinite applications of blockchain t...,nearinfinite applications blockchain technolog...,"[nearinfinite, applications, blockchain, techn...",0,p,1,"[array(['DeFi', 'ORG'], dtype=object)\n array(...","[DeFi, DeFi, DeFi, healthcare, Expanding Oppor...",[],[Pixabay]
478,https://www.globenewswire.com/news-release/202...,2020-06-22,en,La Sierra University to Deploy OMNIQ’s Artific...,\nLa Sierra University to Deploy OMNIQ’s Artif...,"Based on OMNIQ’s AI-Machine Vision technology,...",based omniqs aimachine vision technology percs...,"[based, omniqs, aimachine, vision, technology,...",1,p,1,"[array(['OMNIQ', 'ORG'], dtype=object)\n array...","[OMNIQ, AI-Machine Vision, PERCS, La Sierra Un...",[Fortune 500],[USA]
481,https://www.globenewswire.com/news-release/202...,2022-11-22,en,Global Audio AI Recognition Market to Surpass ...,\nGlobal Audio AI Recognition Market to Surpas...,"November 22, 2022 06:30 ET | : AstuteAnalytica...",november 22 2022 0630 et astuteanalytica india...,"[november, 22, 2022, 0630, et, astuteanalytica...",0,p,1,"[array(['November 22, 2022', 'DATE'], dtype=ob...","[AstuteAnalytica India Pvt, GLOBE NEWSWIRE, AI...",[],"[Noida, INDIA, New Delhi, Nuance, Nuance, Nuan..."


In [ ]:
summarize_pos_df['cleaned_text']

182       Held at the Jameel Arts Centre in Dubai, the c...
200       AI chipsets see greater opportunities in the a...
309       The near-infinite applications of blockchain t...
478       Based on OMNIQ’s AI-Machine Vision technology,...
481       November 22, 2022 06:30 ET | : AstuteAnalytica...
                                ...                        
199623    Abu Dhabi’s Thriving Artificial Intelligence S...
199626    Spotify Introduces an AI DJ for Mobile Devices...
199843    The integration follows Babylon's recent acqui...
200152    The AI Awards 2.0 is an initiative open to all...
200249    HCA, others use Franklin startup’s AI for canc...
Name: cleaned_text, Length: 1156, dtype: object

In [ ]:
pos_text = str(summarize_pos_df['cleaned_text'].tolist())
pos_text[:390]

"['Held at the Jameel Arts Centre in Dubai, the conference was attended by H.E. Omar Sultan Al Olama, UAE Minister for Artificial Intelligence, Digital Economy and Remote Work Applications. The conference marks the first international venture of ‘AI Cures’, the Jameel Clinic’s platform for collaboration that launched in the early months of the COVID-19 pandemic. Fady Jameel, vice chairman"

In [ ]:
type(pos_text)

str

In [ ]:
from summarizer import Summarizer

In [ ]:
model = Summarizer()

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
result = model(pos_text[:200], ratio=0.1)  # Specified with ratio
result

"['Held at the Jameel Arts Centre in Dubai, the conference was attended by H.E. Omar Sultan Al Olama, UAE Minister for Artificial Intelligence, Digital Economy and Remote Work Applications."

In [ ]:
summarize_neg_df = p_df[p_df['entities'].str.contains('business')]
summarize_neg_df.head()

,url,date,language,title,text,cleaned_text,important_words,tokens,topic,predicted_sentiment_yelp_new,sentiment_numeric,entities,organization,products,GPE
324,https://www.bovnews.com/companies/could-genera...,2022-03-07,en,Could generate strong returns? Devon Energy Co...,\n\nCould generate strong returns? Devon Energ...,"Devon Energy Corporation (DVN), Guardforce AI ...",devon energy corporation dvn guardforce ai co ...,"[devon, energy, corporation, dvn, guardforce, ...",5,p,1,"[array(['Devon Energy Corporation', 'ORG'], dt...","[Devon Energy Corporation, Guardforce AI Co. L...",[],[]
1333,https://www.bovnews.com/2022/02/12/great-prote...,2022-02-13,en,Great Protection In An Uncertain Environment: ...,\n\nGreat Protection In An Uncertain Environme...,Great Protection In An Uncertain Environment: ...,great protection uncertain environment guardfo...,"[great, protection, uncertain, environment, gu...",5,p,1,"[array(['Guardforce AI Co. Limited', 'ORG'], d...","[Guardforce AI Co. Limited, CinCor Pharma Inc....",[],[]
1381,https://www.crn.com.au/news/kinetic-its-ai-sav...,2022-03-31,en,Kinetic IT’s AI saves Qantas 200 hours a month...,\n\nKinetic IT’s AI saves Qantas 200 hours a m...,The MSP said its AWS-based ‘Automated Virtual ...,msp said awsbased automated virtual agent ava ...,"[msp, said, awsbased, automated, virtual, agen...",0,p,1,"[array(['MSP', 'ORG'], dtype=object) array(['A...","[MSP, AWS, AVA, Qantas, AVA, AVA, AVA, Qantas,...",[],[Perth]
2271,https://thriveglobal.com/stories/hani-zeini-ex...,2021-01-08,en,Hani Zeini explains how Artificial Intelligenc...,\nHani Zeini explains how Artificial Intellige...,Hani Zeini explains how Artificial Intelligenc...,hani zeini explains artificial intelligence st...,"[hani, zeini, explains, artificial, intelligen...",0,p,1,"[array(['Hani Zeini', 'PERSON'], dtype=object)...","[Artificial Intelligence, The Thrive Global Co...",[JavaScript],[]
2301,https://www.13abc.com/prnewswire/2022/05/17/ar...,2022-05-17,en,Argo AI Begins Driverless Vehicle Operations i...,Argo AI Begins Driverless Vehicle Operations i...,FoodBest Of ClassLatest NewscastsCircle - Coun...,foodbest classlatest newscastscircle country m...,"[foodbest, classlatest, newscastscircle, count...",7,p,1,[array(['ReleasesArgo AI Begins Driverless Veh...,[ReleasesArgo AI Begins Driverless Vehicle Ope...,[],"[Miami, Austin, Washington, D.C., Miami Beach,..."


In [ ]:
neg_text = str(summarize_neg_df['cleaned_text'].tolist())
neg_text[:390]

"['Devon Energy Corporation (DVN), Guardforce AI Co. Limited (GFAI) | BOV News Sign in Home Share Market Snapshot Stock Gestures Trading Guidance Trending Equities Sign in Welcome!Log into your account your username your password Forgot your password? Password recovery Recover your password your email Search Home Share Market Snapshot Stock Gestures Trading Guidance Trending Equities More"

In [ ]:
model = Summarizer()

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
result = model(neg_text[:200], ratio=0.1)  # Specified with ratio
result

"['Devon Energy Corporation (DVN), Guardforce AI Co. Limited (GFAI) | BOV News Sign in Home Share Market Snapshot Stock Gestures Trading Guidance Trending Equities Sign in Welcome!Log into your account"